In [1]:
'''
Tools for blockchain data interaction
'''

import os
import requests
import json
import pandas as pd
from glob import glob
from dotenv import load_dotenv
from web3 import Web3
from hexbytes import HexBytes
from uniswap_universal_router_decoder import RouterCodec

load_dotenv()

# .env variables
ETHERSCAN_API_KEY=os.getenv("ETHERSCAN_API_KEY")
RPC_INFURA_HTTPS_ENDPOINT=os.getenv('RPC_INFURA_HTTPS_ENDPOINT')

rpc_provider = Web3.HTTPProvider(RPC_INFURA_HTTPS_ENDPOINT)
web3 = Web3(provider=rpc_provider)
routerCodec = RouterCodec(w3=web3)


ModuleNotFoundError: No module named 'pandas'

In [2]:
'''
Contract address, tracked events, input data
'''

UNIVERSAL_ROUTER_CONTRACT_ADDRESS="0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD".lower()

UNISWAP_V2_SWAP_EVENT="0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822"
UNISWAP_V3_SWAP_EVENT="0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"
TRACKED_EVENTS=[UNISWAP_V2_SWAP_EVENT, UNISWAP_V3_SWAP_EVENT]

def read_csv_files(paths_array):
    """
    Concatenates all CSV files in the specified path into a single DataFrame.

    Parameters:
    - paths_array (str): Array of paths containing CSV files.

    Returns:
    - pd.DataFrame: Concatenated DataFrame.
    """

    li = []

    if not paths_array:
        raise FileNotFoundError(f"No CSV files found in the specified paths array '{paths_array}'.")

    for file_path in paths_array:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"The specified path '{file_path}' does not exist.")

        df = pd.read_csv(file_path, index_col=None, header=0)
        li.append(df)

    return pd.concat(li, axis=0, ignore_index=True)

transactions_files_paths=glob('sample_data/data/raw/blockchains/eth/transactions/start_block=*/end_block=*/transactions*.csv')
logs_files_paths=glob('sample_data/data/raw/blockchains/eth/logs/start_block=*/end_block=*/logs*.csv')

eth_blockchain_transactions_df_columns=['hash', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'block_timestamp', 'max_fee_per_gas', 'max_priority_fee_per_gas', 'transaction_type']
eth_blockchain_logs_df_columns=['log_index', 'transaction_hash', 'transaction_index', 'block_hash', 'block_number', 'address', 'data', 'topics']

eth_blockchain_transactions_df = read_csv_files(transactions_files_paths)[eth_blockchain_transactions_df_columns].rename(columns={'hash' : 'transaction_hash', 'from_address': 'sender_address'})
eth_blockchain_logs_df = read_csv_files(logs_files_paths)[eth_blockchain_logs_df_columns].rename(columns={'address' : 'event_src_addr'})

In [10]:
universal_router_transactions_df = eth_blockchain_transactions_df[(eth_blockchain_transactions_df['to_address'] == UNIVERSAL_ROUTER_CONTRACT_ADDRESS)].drop(columns='to_address')
# print(universal_router_transactions_df)

universal_router_logs_df = eth_blockchain_logs_df[eth_blockchain_logs_df['transaction_hash'].isin(universal_router_transactions_df['transaction_hash'])]
# print(universal_router_logs_df)

universal_router_tracked_events_logs = universal_router_logs_df[universal_router_logs_df['topics'].str.split(',').apply(lambda topics: topics[0]).isin(TRACKED_EVENTS)]

universal_router_tracked_events_logs

,log_index,transaction_hash,transaction_index,block_hash,block_number,event_src_addr,data,topics
86,86,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,27,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
177,177,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,75,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x5f1002ae7ce00caca60abd017a5aaceb01f09349,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
233,233,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
239,239,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
265,265,0x0ba3bfd593613e19c499b8ba9130ebc95e786d8fa5d6...,103,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x93ba064d66a92ccca2827f688be8f7b352539957,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
...,...,...,...,...,...,...,...,...
255690,398,0x6675f3e6dc788307ff2c7d5f0da5e5f01be4ed791f31...,189,0x0580489a6a124a95d1b077df0f250e4bd81675e1837e...,18257738,0xe8c6c9227491c0a8156a0106a0204d881bb7e531,0xffffffffffffffffffffffffffffffffffffffffffff...,0xc42079f94a6350d7e6235f29174924f928cc2ac818eb...
255741,125,0x223804a66b3809d40b1a3000c4ec822372f4a5b1ccf9...,56,0xfd4f2723407c6c14e4c5757c85bc94cb6f34497387b6...,18257741,0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf,0xffffffffffffffffffffffffffffffffffffffffffff...,0xc42079f94a6350d7e6235f29174924f928cc2ac818eb...
255901,285,0xb11097c3920771919f8f8d00318031ec27d470e7c929...,68,0xfd4f2723407c6c14e4c5757c85bc94cb6f34497387b6...,18257741,0xfbfdde37a8c5e24f5b23c33e4bb11a32eb70bf02,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
255978,41,0x9e82480ce5bbde3de5c6b55e5c5c9f8993a77fa463e5...,33,0x485ae9b8b725a22bec6fee2cdb06295fd78eca98a4d9...,18257742,0xae30f13675a1b58c313663f066c43e6190457a76,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...


In [4]:
# Universal Router tracked events logs df
universal_router_tracked_events_logs.head(3)

,log_index,transaction_hash,transaction_index,block_hash,block_number,event_src_addr,data,topics
86,86,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,27,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
177,177,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,75,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x5f1002ae7ce00caca60abd017a5aaceb01f09349,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...
233,233,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...


In [5]:
# Universal Router transactions df
# print(universal_router_transactions_df.head(3).iloc(1)[0][75]) # hash
# print(universal_router_transactions_df.head(3).iloc(1)[10][75]) # input
universal_router_transactions_df.head(3)

,transaction_hash,sender_address,value,gas,gas_price,input,block_timestamp,max_fee_per_gas,max_priority_fee_per_gas,transaction_type
27,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,0x6d580849c3b098aa559479cd68e4c86664fc5b77,30000000000000000,222069,10811754905,0x24856bc3000000000000000000000000000000000000...,1696179587,1.207127e+10,2.000000e+09,2
75,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,0x29131059129132b59249848b4505ca4d907807c9,0,252950,9111754905,0x3593564c000000000000000000000000000000000000...,1696179587,1.444910e+10,3.000000e+08,2
96,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,489536,8911754905,0x3593564c000000000000000000000000000000000000...,1696179587,1.088466e+10,1.000000e+08,2


In [8]:
'''
Join two sources (dataframes)
'''

merged_df = pd.merge(universal_router_tracked_events_logs, universal_router_transactions_df, on='transaction_hash')
merged_df

,log_index,transaction_hash,transaction_index,block_hash,block_number,event_src_addr,data,topics,sender_address,value,gas,gas_price,input,block_timestamp,max_fee_per_gas,max_priority_fee_per_gas,transaction_type
0,86,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,27,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x6d580849c3b098aa559479cd68e4c86664fc5b77,30000000000000000,222069,10811754905,0x24856bc3000000000000000000000000000000000000...,1696179587,1.207127e+10,2.000000e+09,2
1,177,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,75,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x5f1002ae7ce00caca60abd017a5aaceb01f09349,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x29131059129132b59249848b4505ca4d907807c9,0,252950,9111754905,0x3593564c000000000000000000000000000000000000...,1696179587,1.444910e+10,3.000000e+08,2
2,233,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,489536,8911754905,0x3593564c000000000000000000000000000000000000...,1696179587,1.088466e+10,1.000000e+08,2
3,239,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,489536,8911754905,0x3593564c000000000000000000000000000000000000...,1696179587,1.088466e+10,1.000000e+08,2
4,265,0x0ba3bfd593613e19c499b8ba9130ebc95e786d8fa5d6...,103,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x93ba064d66a92ccca2827f688be8f7b352539957,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x6e132c4c256255174e6fcf72c549bc58d737da45,10000000000000000,163526,8911754905,0x3593564c000000000000000000000000000000000000...,1696179587,1.160165e+10,1.000000e+08,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8234,398,0x6675f3e6dc788307ff2c7d5f0da5e5f01be4ed791f31...,189,0x0580489a6a124a95d1b077df0f250e4bd81675e1837e...,18257738,0xe8c6c9227491c0a8156a0106a0204d881bb7e531,0xffffffffffffffffffffffffffffffffffffffffffff...,0xc42079f94a6350d7e6235f29174924f928cc2ac818eb...,0x3c4e08d4b09ef4b12e6a8ab0b0c8bbc289b36673,0,314229,6344924886,0x3593564c000000000000000000000000000000000000...,1696188071,8.578095e+09,1.000000e+07,2
8235,125,0x223804a66b3809d40b1a3000c4ec822372f4a5b1ccf9...,56,0xfd4f2723407c6c14e4c5757c85bc94cb6f34497387b6...,18257741,0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf,0xffffffffffffffffffffffffffffffffffffffffffff...,0xc42079f94a6350d7e6235f29174924f928cc2ac818eb...,0xfe9d9a0e27a59a3f33a4ede0894f72d6de2017a8,0,167059,6230066635,0x3593564c000000000000000000000000000000000000...,1696188107,1.106937e+10,3.000000e+08,2
8236,285,0xb11097c3920771919f8f8d00318031ec27d470e7c929...,68,0xfd4f2723407c6c14e4c5757c85bc94cb6f34497387b6...,18257741,0xfbfdde37a8c5e24f5b23c33e4bb11a32eb70bf02,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x35f32de5c007d8e24c6ac0bf4573e73e52fc4602,15000000000000000,224421,6066658618,0x3593564c000000000000000000000000000000000000...,1696188107,NaN,NaN,0
8237,41,0x9e82480ce5bbde3de5c6b55e5c5c9f8993a77fa463e5...,33,0x485ae9b8b725a22bec6fee2cdb06295fd78eca98a4d9...,18257742,0xae30f13675a1b58c313663f066c43e6190457a76,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0fe463fbb8ec8db3a8478356a7312e57282f30cd,0,292882,7350000000,0x3593564c00000000000000000000000000000000000

In [11]:
'''
Decrypt transaction 'input' and retrieve amountIn, amountOutMin, path (convert if event == UNISWAP V3)
'''

def check_array_size(arr):
    if len(arr) > 1:
        raise ValueError("More than 1 swap event in single record")

# print(merged_df['event_src_addr'].value_counts())

merged_df_sample = merged_df.head(10)

CMD_V2_SWAP_EXACT_IN = 'V2_SWAP_EXACT_IN'
CMD_V2_SWAP_EXACT_OUT = 'V2_SWAP_EXACT_OUT'
CMD_V3_SWAP_EXACT_IN = 'V3_SWAP_EXACT_IN'
CMD_V3_SWAP_EXACT_OUT = 'V3_SWAP_EXACT_OUT'

FILTER_CMDS = (CMD_V2_SWAP_EXACT_IN, CMD_V2_SWAP_EXACT_OUT, CMD_V3_SWAP_EXACT_IN, CMD_V3_SWAP_EXACT_OUT)

for index, row in merged_df_sample.iterrows():

    decoded_trx_input = routerCodec.decode.function_input(row['input'])

    filtered_data = [item for item in decoded_trx_input[1]['inputs'] if any(cmd_name in str(item[0]) for cmd_name in FILTER_CMDS)]

    check_array_size(filtered_data)

    function, params = filtered_data[0]
    # print("--------")
    # print(row['transaction_hash'])
    # print(decoded_trx_input)
    # print(params['path'])

    CMD_FUNCTION = str(function)

    if CMD_V2_SWAP_EXACT_IN in CMD_FUNCTION:
        merged_df_sample.loc[index, 'command_identifier'] = CMD_V2_SWAP_EXACT_IN
        merged_df_sample.loc[index, 'token_address_in'] = params['path'][0]
        merged_df_sample.loc[index, 'token_address_out'] = params['path'][1]
        merged_df_sample.loc[index, 'swap_amount_in'] = params['amountIn']
        merged_df_sample.loc[index, 'swap_amount_out_min'] = params['amountOutMin']
    elif CMD_V2_SWAP_EXACT_OUT in CMD_FUNCTION:
        merged_df_sample.loc[index, 'command_identifier'] = CMD_V2_SWAP_EXACT_OUT
        merged_df_sample.loc[index, 'token_address_in'] = params['path'][0]
        merged_df_sample.loc[index, 'token_address_out'] = params['path'][1]
        merged_df_sample.loc[index, 'swap_amount_in_max'] = params['amountInMax']
        merged_df_sample.loc[index, 'swap_amount_out'] = params['amountOut']
    elif CMD_V3_SWAP_EXACT_IN in CMD_FUNCTION:
        decoded_path = routerCodec.decode.v3_path(CMD_V3_SWAP_EXACT_IN, params['path'])
        merged_df_sample.loc[index, 'command_identifier'] = CMD_V3_SWAP_EXACT_IN
        merged_df_sample.loc[index, 'token_address_in'] = decoded_path[0]
        merged_df_sample.loc[index, 'token_address_out'] = decoded_path[2]
        merged_df_sample.loc[index, 'swap_amount_in'] = params['amountIn']
        merged_df_sample.loc[index, 'swap_amount_out_min'] = params['amountOutMin']
    elif CMD_V3_SWAP_EXACT_OUT in CMD_FUNCTION:
        decoded_path = routerCodec.decode.v3_path(CMD_V3_SWAP_EXACT_OUT, params['path'])
        merged_df_sample.loc[index, 'command_identifier'] = CMD_V3_SWAP_EXACT_OUT
        merged_df_sample.loc[index, 'token_address_in'] = decoded_path[2]
        merged_df_sample.loc[index, 'token_address_out'] = decoded_path[0]
        merged_df_sample.loc[index, 'swap_amount_in_max'] = params['amountInMax']
        merged_df_sample.loc[index, 'swap_amount_out'] = params['amountOut']


merged_df_sample = merged_df_sample.drop(columns='input')
merged_df_sample

C:\Users\mkrol\AppData\Local\Temp\ipykernel_33612\2526627454.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_sample.loc[index, 'command_identifier'] = CMD_V2_SWAP_EXACT_IN
C:\Users\mkrol\AppData\Local\Temp\ipykernel_33612\2526627454.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_sample.loc[index, 'token_address_in'] = params['path'][0]
C:\Users\mkrol\AppData\Local\Temp\ipykernel_33612\2526627454.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,log_index,transaction_hash,transaction_index,block_hash,block_number,event_src_addr,data,topics,sender_address,value,...,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,command_identifier,token_address_in,token_address_out,swap_amount_in,swap_amount_out_min,swap_amount_in_max,swap_amount_out
0,86,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,27,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x6d580849c3b098aa559479cd68e4c86664fc5b77,30000000000000000,...,1.207127e+10,2.000000e+09,2,V2_SWAP_EXACT_IN,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xAb320334950B365379bF6f505C7bc8763ce6641c,30000000000000000.0,120600693930419.0,NaN,NaN
1,177,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,75,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x5f1002ae7ce00caca60abd017a5aaceb01f09349,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x29131059129132b59249848b4505ca4d907807c9,0,...,1.444910e+10,3.000000e+08,2,V2_SWAP_EXACT_IN,0xc94862ed8F5f368252f12821b3DD28b00BCDf5a5,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2120777171806358.0,233517830261887072.0,NaN,NaN
2,233,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,...,1.088466e+10,1.000000e+08,2,V2_SWAP_EXACT_IN,0x2b8DDAf520f38c23BB638766666c6f6952b2d064,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,261937303117650344374,30273438111052900.0,NaN,NaN
3,239,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,...,1.088466e+10,1.000000e+08,2,V2_SWAP_EXACT_IN,0x2b8DDAf520f38c23BB638766666c6f6952b2d064,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,261937303117650344374,30273438111052900.0,NaN,NaN
4,265,0x0ba3bfd593613e19c499b8ba9130ebc95e786d8fa5d6...,103,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x93ba064d66a92ccca2827f688be8f7b352539957,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x6e132c4c256255174e6fcf72c549bc58d737da45,10000000000000000,...,1.160165e+10,1.000000e+08,2,V2_SWAP_EXACT_IN,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xDFd540B8A14FaB2cf02a67fAcdc61B6eE5Fee577,10000000000000000,110501615614802257470151,NaN,NaN
5,324,0x174f29b21bf123c08b946d4d58634489880229bed28c...,128,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x8915315d1bddbc3becac4fa70b17d4881d1761f8,0,...,1.024113e+10,1.000000e+07,2,V2_SWAP_EXACT_IN,0xAb320334950B365379bF6f505C7bc8763ce6641c,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,145259625212904,28339416433012211,NaN,NaN
6,329,0x8a31eb27e80039aa6be10c2110d5293099856a02589d...,129,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x977c5fcf7a552d38adcde4f41025956855497c6d,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x4e6755ace70972a78b1c173ac31c0cebac0eabb7,0,...,1.024113e+10,1.000000e+07,2,V2_SWAP_EXACT_OUT,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xfF836A5821E69066c87E268bC51b849FaB94240C,NaN,NaN,1.796608e+16,500000000000000000000000
7,333,0x19e35cc0118c78c94f35ba460835457ec4f6a9be1025...,131,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x88395224c3709be9a700624bd9246459585c56b0,0x00000000000000000000000000000000000000000000...,0xc42079f94a6350d7e6235f29174

In [288]:
'''
Retrieve token name and symbol from provided contract address

This block is used for retrieving token IN and token OUT name/symbol.

It requires API call to etherscan.io API, which is not free (free plan limit: 5 calls/sec)

TODO 1 -> Consider if this operation is needed (time consuming + expensive API costs)

TODO 2 -> Consider following solution: redis cache + postgreSQL + kafka (kafka for real-time backup and redis cache update)

'''

def get_contract_abi(contract_address, api_key) ->dict:
    """
        Get contract ABI for provided contract
    """
    etherscan_call = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={api_key}"
    r = requests.get(url = etherscan_call)
    contract_abi =  json.loads(json.loads(r.text)['result'])
    return contract_abi

for index, row in merged_df_sample.iterrows():
    token_address_in = row['token_address_in']
    token_address_in_abi = get_contract_abi(contract_address=token_address_in, api_key=ETHERSCAN_API_KEY)
    token_address_out = row['token_address_out']
    token_address_out_abi = get_contract_abi(contract_address=token_address_out, api_key=ETHERSCAN_API_KEY)

    contract_from = web3.eth.contract(address=token_address_in, abi=token_address_in_abi)
    contract_to = web3.eth.contract(address=token_address_out, abi=token_address_out_abi)

    token_in_name = contract_from.functions.name().call()
    token_in_symbol = contract_from.functions.symbol().call()

    token_out_name = contract_to.functions.name().call()
    token_out_symbol = contract_to.functions.symbol().call()

    # print(f"token IN: ({token_in_name}, {token_in_symbol})")
    # print(f"token OUT: ({token_out_name}, {token_out_symbol})")

    merged_df_sample.loc[index, 'token_in_name'] = token_in_name
    merged_df_sample.loc[index, 'token_in_symbol'] = token_in_symbol
    merged_df_sample.loc[index, 'token_out_name'] = token_out_name
    merged_df_sample.loc[index, 'token_out_symbol'] = token_out_symbol

    # ((token_in_name, token_in_symbol),(token_out_name, token_out_symbol))

merged_df_sample

,log_index,transaction_hash,transaction_index,block_hash,block_number,event_src_addr,data,topics,sender_address,value,...,token_address_in,token_address_out,swap_amount_in,swap_amount_out_min,swap_amount_in_max,swap_amount_out,token_in_name,token_in_symbol,token_out_name,token_out_symbol
0,86,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,27,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x6d580849c3b098aa559479cd68e4c86664fc5b77,30000000000000000,...,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xAb320334950B365379bF6f505C7bc8763ce6641c,30000000000000000.0,120600693930419.0,NaN,NaN,Wrapped Ether,WETH,Pepe of Wallstreet,POW
1,177,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,75,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x5f1002ae7ce00caca60abd017a5aaceb01f09349,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x29131059129132b59249848b4505ca4d907807c9,0,...,0xc94862ed8F5f368252f12821b3DD28b00BCDf5a5,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2120777171806358.0,233517830261887072.0,NaN,NaN,Sendor,SENDOR,Wrapped Ether,WETH
2,233,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,...,0x2b8DDAf520f38c23BB638766666c6f6952b2d064,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,261937303117650344374,30273438111052900.0,NaN,NaN,FriendX,FRIENDX,Wrapped Ether,WETH
3,239,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,...,0x2b8DDAf520f38c23BB638766666c6f6952b2d064,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,261937303117650344374,30273438111052900.0,NaN,NaN,FriendX,FRIENDX,Wrapped Ether,WETH
4,265,0x0ba3bfd593613e19c499b8ba9130ebc95e786d8fa5d6...,103,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x93ba064d66a92ccca2827f688be8f7b352539957,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x6e132c4c256255174e6fcf72c549bc58d737da45,10000000000000000,...,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xDFd540B8A14FaB2cf02a67fAcdc61B6eE5Fee577,10000000000000000,110501615614802257470151,NaN,NaN,Wrapped Ether,WETH,SerBridge,SBDG
5,324,0x174f29b21bf123c08b946d4d58634489880229bed28c...,128,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x8915315d1bddbc3becac4fa70b17d4881d1761f8,0,...,0xAb320334950B365379bF6f505C7bc8763ce6641c,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,145259625212904,28339416433012211,NaN,NaN,Pepe of Wallstreet,POW,Wrapped Ether,WETH
6,329,0x8a31eb27e80039aa6be10c2110d5293099856a02589d...,129,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x977c5fcf7a552d38adcde4f41025956855497c6d,0x00000000000000000000000000000000000000000000...,0xd78ad95fa46c994b6551d0da85fc275fe613ce37657f...,0x4e6755ace70972a78b1c173ac31c0cebac0eabb7,0,...,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xfF836A5821E69066c87E268bC51b849FaB94240C,NaN,NaN,1.796608e+16,500000000000000000000000,Wrapped Ether,WETH,Real Smurf Cat,шайлушай
7,333,0x19e35cc0118c78c94f35ba460835457ec4f6a9be1025...,131,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x88395224c3709be9a700624bd9246459585c56b0,0x00000000000000000000000000000000000000000000...,0xc42079f94a6350d7e6235f29174924f928cc2ac818eb...,0x218e67ace2d1e5bec42721fa0f6b60aa2baf591a,500

In [294]:
'''
Decrypt 'data' from event logs (fields depends if event == V2 or V3)

TODO -> redis cache + postgreSQL + kafka (kafka for real-time backup and redis cache update) for ABI retrieving

'''

for index, row in merged_df_sample.iterrows():

    address = web3.to_checksum_address(row['event_src_addr'])
    address_abi=get_contract_abi(address, ETHERSCAN_API_KEY)

    pool_contract = web3.eth.contract(address=address, abi=address_abi)

    decoded_event = pool_contract.events.Swap().process_log({
        'data': row['data'],
        'topics': [HexBytes(topic) for topic in row['topics'].split(",")],
        'logIndex': row['log_index'],
        'transactionIndex': row['transaction_index'],
        'transactionHash': row['transaction_hash'],
        'address': row['event_src_addr'],
        'blockHash': row['block_hash'],
        'blockNumber': row['block_number']
    })

    cmd_identifier = row['command_identifier']

    if cmd_identifier in (CMD_V2_SWAP_EXACT_IN, CMD_V2_SWAP_EXACT_OUT):
        merged_df_sample.loc[index, 'v2_amount0In'] = decoded_event['args']['amount0In']
        merged_df_sample.loc[index, 'v2_amount1In'] = decoded_event['args']['amount1In']
        merged_df_sample.loc[index, 'v2_amount0Out'] = decoded_event['args']['amount0Out']
        merged_df_sample.loc[index, 'v2_amount1Out'] = decoded_event['args']['amount1Out']
    elif cmd_identifier in (CMD_V3_SWAP_EXACT_IN, CMD_V3_SWAP_EXACT_OUT):
        merged_df_sample.loc[index, 'v3_amount0'] = decoded_event['args']['amount0']
        merged_df_sample.loc[index, 'v3_amount1'] = decoded_event['args']['amount1']
        merged_df_sample.loc[index, 'v3_sqrtPriceX96'] = decoded_event['args']['sqrtPriceX96']
        merged_df_sample.loc[index, 'v3_liquidity'] = decoded_event['args']['liquidity']
        merged_df_sample.loc[index, 'v3_tick'] = decoded_event['args']['tick']

final_df = merged_df_sample.drop(columns=['data', 'topics'])
final_df

,log_index,transaction_hash,transaction_index,block_hash,block_number,event_src_addr,sender_address,value,gas,gas_price,...,token_out_symbol,v2_amount0In,v2_amount1In,v2_amount0Out,v2_amount1Out,v3_amount0,v3_amount1,v3_sqrtPriceX96,v3_liquidity,v3_tick
0,86,0x53cf14b25c5d1316b53f4b8e73e32fcfb4232bd73c2b...,27,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x6d580849c3b098aa559479cd68e4c86664fc5b77,30000000000000000,222069,10811754905,...,POW,0,3.000000e+16,142786387236868,0,NaN,NaN,NaN,NaN,NaN
1,177,0xbbcddf18079d76b163f269c0853c2ff1dbac05e69b9d...,75,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x5f1002ae7ce00caca60abd017a5aaceb01f09349,0x29131059129132b59249848b4505ca4d907807c9,0,252950,9111754905,...,WETH,0,2.099569e+15,239893045456507365,0,NaN,NaN,NaN,NaN,NaN
2,233,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,489536,8911754905,...,WETH,120218946367444723413,0.000000e+00,0,15299531338126664,NaN,NaN,NaN,NaN,NaN
3,239,0x6272d99f7278c79db9483ef4a6e1978a655bca2f3fa8...,96,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xddcc52a3318293dcb02bb631c1915e013c7dc34c,0x0c4496358f5e2da56462bdddc450c1b2754f6852,0,489536,8911754905,...,WETH,248840437961767827156,0.000000e+00,0,31583332876089604,NaN,NaN,NaN,NaN,NaN
4,265,0x0ba3bfd593613e19c499b8ba9130ebc95e786d8fa5d6...,103,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x93ba064d66a92ccca2827f688be8f7b352539957,0x6e132c4c256255174e6fcf72c549bc58d737da45,10000000000000000,163526,8911754905,...,SBDG,10000000000000000,0.000000e+00,0,111054123692876268757502,NaN,NaN,NaN,NaN,NaN
5,324,0x174f29b21bf123c08b946d4d58634489880229bed28c...,128,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0xcfc5aeac89e573b7b19c2bada006dac582b34ea1,0x8915315d1bddbc3becac4fa70b17d4881d1761f8,0,275707,8821754905,...,WETH,143807028960775,0.000000e+00,0,32263163753107085,NaN,NaN,NaN,NaN,NaN
6,329,0x8a31eb27e80039aa6be10c2110d5293099856a02589d...,129,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x977c5fcf7a552d38adcde4f41025956855497c6d,0x4e6755ace70972a78b1c173ac31c0cebac0eabb7,0,150976,8821754905,...,шайлушай,17110551725827927,0.000000e+00,0,500000000000000000172686,NaN,NaN,NaN,NaN,NaN
7,333,0x19e35cc0118c78c94f35ba460835457ec4f6a9be1025...,131,0x27bf13cffca28a5c47c4ca753ac02ea72a431d123235...,18257034,0x88395224c3709be9a700624bd9246459585c56b0,0x218e67ace2d1e5bec42721fa0f6b60aa2baf591a,50000000000000000,166431,8821754905,...,FUND,NaN,NaN,NaN,NaN,5.000000e+16,-5.212066e+11,256955053069631085013386804,1.551847e+17,-114630.0
8,80,0x0cac70512b57cc78b69e1f08f33c9d693489173fb5f8...,25,0x41fb348c5b21cd8915bd5a20b0b9f25c5380153f31b7...,18257035,0x4ea15a1e4f3955c798722e33031ef818284e1b03,0xf01f3ad7e79cc0457e3d5b43c11fa19ebb941cee,10000000000000000,258868,11087797373,...,COPYBOT,0,1.000000e+16,140146474806944067630,0,NaN,NaN,NaN,NaN,NaN
9,123,0x03eade469c79b660243b1962fe830e1530ebfa080797...,55,0x41fb348c5b21cd8915bd5a20b0b9f25c5380153f31b7...,18257035,0xc32c2d9768919deaee60f7eb163a293c6eb81be8,0x77b9020da34704e0cd6bf7220d252cb15346b483,1000000000000000000,225025,9387797373,...,OCISLY,0,1.000000e+18,81080790233001832,0,NaN,NaN,NaN,NaN,NaN


In [292]:
'''
Save data
'''
final_df.to_csv('ur_transactions.csv')